<img src="images/BDG_LOGO.png" alt="drawing" align="right" width="200"/>

# H2020 RIA BigDataGrapes - Predictive Data Analytics (T4.3)

### This deliverable (D4.3) presents how to train machine learning models with the BigDataGrapes distributed processing architecture. In particular, we present how to train classifiers with MLLib (https://spark.apache.org/mllib/).

In [1]:
from pyspark import SparkContext

In [2]:
import math
import urllib
import random
import numpy as np
import pydoop.hdfs as hdfs

from numpy import array
from pyspark.mllib.regression import LabeledPoint

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

## Connection to the BDG Apache Spark

In [4]:
# standalone mode below
#sc = SparkContext(appName="Classification-WineDataset", master="master[*]")

# distributed mode below
sc = SparkContext(appName="Classification-WineDataset", master="spark://spark-master:7077")

# setting logger level
sc.setLogLevel("ERROR")

# Classification on a real dataset

## We employ a subset of the Red Wine Quality Dataset made available on Kaggle (https://www.kaggle.com/piyushgoyal443/red-wine-dataset/discussion).

### The dataset is related to red variants of the Portuguese "Vinho Verde" wine. For more details, consult: http://www.vinhoverde.pt/en/ or the reference [Cortez et al., 2009]. Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

### The dataset can be viewed as classification or regression tasks. The classes are ordered and not balanced (e.g. there are munch more normal wines than excellent or poor ones). Outlier detection algorithms could be used to detect the few excellent or poor wines. Also, we are not sure if all input variables are relevant. So it could be interesting to test feature selection methods.

### Dataset Properties

#### Number of Instances: red wine - 1599.
#### Number of Attributes: 11 + output attribute
#### Note: several of the attributes may be correlated, thus it makes sense to apply some sort of feature selection.

### Attribute information:

#### Input variables: (based on physicochemical tests): fixed acidity (tartaric acid - g / dm^3), volatile acidity (acetic acid - g / dm^3), citric acid (g / dm^3), residual sugar (g / dm^3), chlorides (sodium chloride - g / dm^3, free sulfur dioxide (mg / dm^3), total sulfur dioxide (mg / dm^3), density (g / cm^3), pH, sulphates (potassium sulphate - g / dm3), alcohol (% by volume)

### Description of attributes:

1. fixed acidity: most acids involved with wine or fixed or nonvolatile (do not evaporate readily)
2. volatile acidity: the amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste
3. citric acid: found in small quantities, citric acid can add 'freshness' and flavor to wines
4. residual sugar: the amount of sugar remaining after fermentation stops, it's rare to find wines with less than 1 gram/liter and wines with greater than 45 grams/liter are considered sweet
5. chlorides: the amount of salt in the wine
6. free sulfur dioxide: the free form of SO2 exists in equilibrium between molecular SO2 (as a dissolved gas) and bisulfite ion; it prevents microbial growth and the oxidation of wine
7. total sulfur dioxide: amount of free and bound forms of S02; in low concentrations, SO2 is mostly undetectable in wine, but at free SO2 concentrations over 50 ppm, SO2 becomes evident in the nose and taste of wine
8. density: the density of water is close to that of water depending on the percent alcohol and sugar content
9. pH: describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4 on the pH scale
10. sulphates: a wine additive which can contribute to sulfur dioxide gas (S02) levels, wich acts as an antimicrobial and antioxidant
11. alcohol: the percent alcohol content of the wine

#### Missing Attribute Values: None

#### Output variable (based on sensory data): quality (score between 0 and 10)

### For more information:

*  [Cortez et al., 2009] P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. **Modeling wine preferences by data mining from physicochemical properties**. In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

## Storing the dataset on Apache HDFS and Reading it with Apache Spark (creating RDDs)

In [5]:
# data format:
#   "fixed.acidity","volatile.acidity","citric.acid","residual.sugar","chlorides",
#     "free.sulfur.dioxide","total.sulfur.dioxide","density","pH","sulphates","alcohol","quality"

# train data
path_on_disk = "datasets/wineQualityReds-train.csv"
path_on_hdfs = "hdfs://namenode:8020/user/root/wineQualityReds-train01.csv"
# hdfs.put(path_on_disk, path_on_hdfs)
train_data = sc.textFile(path_on_hdfs)

# train data
test_path_on_disk = "datasets/wineQualityReds-test.csv"
test_path_on_hdfs = "hdfs://namenode:8020/user/root/wineQualityReds-test01.csv"
# hdfs.put(test_path_on_disk, test_path_on_hdfs)
test_data = sc.textFile(test_path_on_hdfs)

In [6]:
# parsing data.
# first 11 fields, features
# last field, quality

def parse_interaction(line):
    line_split = line.split(",")
    features = line_split[0:11]
    quality = line_split[11]
    return LabeledPoint(float(quality), array([float(x) for x in features]))

In [7]:
parsed_train_data = train_data.map(parse_interaction)
parsed_test_data = test_data.map(parse_interaction)

# Multi-label Classification

## Now training one Classifier.

### In this task, each grade of quality {1,2,...,10} is a specific class to guess.

### We also measure the time needed by MLLib to train it.

In [8]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

%time model = LogisticRegressionWithLBFGS.train(parsed_train_data, iterations=1, numClasses=10)

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 8.3 s


In [9]:
labels_and_preds = parsed_test_data.map(lambda p: (p.label, model.predict(p.features)))

## Evaluation of the performance of the classifier (Accuracy)

In [10]:
%time test_accuracy = labels_and_preds.filter(lambda (v, p): v == p).count() / float(test_data.count())
print "Accuracy on test data is {}".format(round(test_accuracy, 4))

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 436 ms
Accuracy on test data is 0.4361


## Disconnection from the BDG Apache Spark

In [11]:
sc.stop()